# Python for Data Analytics - Exercises 2 #

# Writing Python functions – Understanding Sampling #

## Objectives ##

* To gain an intuitive understanding of sample statistics.

## 1. Background

In this notebook class you’ll be using data that represents the weights and lengths of male and female **Bengal tigers**. In order to build classifiers we are going to want to be able to estimate the [means](https://en.wikipedia.org/wiki/Mean) (i.e. averages), [variances](https://en.wikipedia.org/wiki/Variance) and [covariance](https://en.wikipedia.org/wiki/Covariance) of these measurements. In order to discover the true means and variances we’d have to catch and measure every one of the roughly 2,500 tigers that are thought to exist – a project that is clearly impractical. So, instead we will have to be satisfied with an **estimate** of the true mean and variance that we generate by looking at the statistics of a small **sample** of tigers (e.g. we might catch and measure 20 animals). This is generally always going to be the case: we will be interested in the statistics of a large population, but we will have to make do with estimates derived from a smaller – often much smaller – sample.

**In Part 1 of this notebook** we will be using Python to find out what happens when we try to compute estimates of statistics using small samples. We will see how the reliability of the estimates improves as the sample size increases. We will also see how estimates of variance and covariance can be biased (i.e. systematically too small or too large) if we don’t handle the sample correctly.

**Part 2 consists of some little Python programming challenges.** 

# PART 1 – Sample Statistics

## 2. Loading and understanding the data

The tiger population data we are using has been made up (i.e. no-one really knows the weights and lengths of every living Bengal tiger) but it has been generated to be consistent with the ranges of weight and length that are quoted on Wikipedia. It has also been engineered so that the population statistics are nice round numbers, just to make things a little clearer.

(**Remember, to execute a cell, first click on it so that it is in focus and then click the run button in the tool bar, or ctrl-enter**).

In [ ]:
import numpy as np
female_tigers = np.loadtxt(open("files/female_tigers.txt","rb"))
male_tigers = np.loadtxt(open("files/male_tigers.txt","rb"))

You will now have two matrices each with 1,250 columns and two rows. The columns represent the tigers (1,250 of each gender). The first row gives the weight (in kg) and the second the length (in cm).     

In [ ]:
print(female_tigers.shape)
print(female_tigers)

We have access to the data for all the tigers so we can compute the true mean and variance of the weight and length features. For example, for the female tigers try,

In [ ]:
print(np.mean(female_tigers, axis=1))
print(np.var(female_tigers, axis=1))

We are now going to imagine that we don’t know this truth but need to estimate it from a small sample of tigers that have been caught, weighed and measured.

## 3. Sampling the data

Imagine that you have gone out and caught 20 female tigers and measured them. You’d now know the measurements of 20 tigers of the 1250 that exist. Let’s simulate this experiment by taking the first 20 of the female tigers from our complete population data  

In [ ]:
sample = female_tigers[:, 1:20]

Let’s compute the mean and variance of both the length and weight features in our sample,  

In [ ]:
print(np.mean(sample, axis=1))
print(np.var(sample, axis=1))

Notice how these estimates are not the same as the true mean and variance. But how close to the real values are they likely to be?

## 4. Repeated sampling

If we repeated our experiment and caught 20 different tigers we would get a different set of measurement and hence a different estimate of the mean and variance. Using Python we can simulate running this experiment 1000’s of times and getting 1000’s of different estimates of the mean and variance. We can then plot a histogram of these estimates and compare them to the true values (which we wouldn’t know in reality but which we computed above by using all the data).

So we will now repeat the experiment. In order to catch 20 different tigers we use the random.choice function in Python's numpy library. This function can return randomly chosen integers between 0 and some maximum value. It can be instructed to select them 'without replacement' i.e. so that the same number cannot be chosen twice. We will use this to make an array of random indices and then use this array to select random columns from the female_tiger data matrix.

In [ ]:
chosen = np.random.choice(female_tigers.shape[1], 20, replace=False)
sample = female_tigers[:, chosen]
print(sample)

Now, compute the sample mean and variance for the tiger weights again and see that the result has changed.

In [ ]:
print(np.mean(sample[1,:]))
print(np.var(sample[1,:]))

Run the above two cells over and over again. Note how each time you get a different mean and variance estimate.

## 5. Writing a sampling function

We are now going to wrap our sampling and mean and variance calculations into an easily reusable function.

The function’s inputs should be the complete population data and the sample size; the output should be the mean and variance estimates, i.e. the first line of the function should look like this,

    def calc_sample_stats(data, N):
 
and the return will be
         
    return mean_estimate, var_estimate

The full code is below:

In [ ]:
def calc_sample_stats(data, N):
    chosen = np.random.choice(data.shape[1], N, replace=False)
    sample = data[:, chosen]
    mean_estimate = np.mean(sample, axis=1)
    var_estimate = np.var(sample, axis=1)
    return mean_estimate, var_estimate

print(calc_sample_stats(female_tigers, 20))

Test the function the using different values of N in the cell below,

In [ ]:
# Insert multiple test calls for the above function by giving N different values

Now we will write another function called do_sampling. This script is going to call your calc_sample_stats function lots of times by using a loop. It will save the results of each run in a python array.

In [ ]:
def do_sampling(data, sample_size, n_repeats):
    sample_means = np.zeros((2,n_repeats))
    sample_vars = np.zeros((2,n_repeats))
    for i in range(n_repeats): 
        sample_means[:,i],sample_vars[:,i]=calc_sample_stats(female_tigers, sample_size)
    return sample_means, sample_vars

sample_means, sample_vars = do_sampling(female_tigers, 20, 10)

We can now plot a histogram of the results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
sample_means, sample_vars = do_sampling(female_tigers, 10, 10000)
plt.hist(sample_means[0,:], 100);

or    

In [ ]:
plt.hist(sample_means[1,:], 100);

How does the distribution of the sample means compare with the true mean value?

To make a histogram of the variance estimates,

In [ ]:
plt.hist(sample_vars[0,:], 100);

or

In [ ]:
plt.hist(sample_vars[1,:], 100);

How does the distribution of the sample variance compare with the true variance?

Calculate the average of the mean estimates, i.e. mean(sample_mean). Are the estimates (on average) the same as the true mean value? Do the same for the variance estimates. Are the variance estimates on average the same as the true variance?

In [ ]:
print(np.mean(sample_vars, axis=1))

## 6. Notes on bias

You should have noticed in the previous section that the variance estimates were on average lower than the true variance. The estimate is biased. You can correct for this bias by multiplying by N/N-1 or, alternatively, by dividing by N-1 instead of N when you do the original variance calculation. 

(Why does the sample variance under-estimate the true variance? Informal explanation: The point to remember is that the variance is the average squared distance of points from the true mean. When we take our sample we form an estimate of the mean. So when we estimate variance we are computing the distance of points to the estimated mean not the true mean. The estimated mean is going to be at the centre of our sample and will always be closer to the sample points than the true mean. This is seen clearly by taking the extreme case of a sample size of 1. For a single point the sample mean will always be the sample itself, i.e. the distance to the sample mean is 0 so the sample variance is 0 --- which is clearly a severe underestimate of the true variance. )

## 7. Experimenting with the sample size

Now let’s experiment with different sample sizes and see how changing the sample size changes the distribution of the estimates of mean and variance.

We will use sample sizes of 2, 5, 10, 20, 40 and 100.

Modify your do_sampling code by wrapping the sampling loop in an outer loop that sets the sample size. We will want to be able to compare the histograms that are generated for each sample size, so use the subplot command to send each histogram to a different subplot in the same figure (i.e. same window). Write the code for this in the empty cell below.

What do you notice about how the distribution of the estimates changes as the sample size increases?

## 8. Classifying

Compare the histogram of weights of the female and male tigers for the entire population. A simple classifier could choose a threshold weight W and then if a tiger weighs more than W it would label it as male and if it weighs less than W it would label it as female. Looking at the histograms, what would be the best choice for the weight W?

Now take a sample of 100 female tigers and 100 male tigers. Plot histograms using these small samples. Looking at the histograms choose a threshold weight again. Is the threshold chosen using the small sample the same as the threshold you chose when using knowledge of the full set of data?

In [ ]:
# Plot a histogram of all the data

In [ ]:
# Plot a histogram of the sampled data

As we will see later in the tutorial, having insufficient 'training data' can lead to poor choices when designing classifiers.

# PART 2 – Programming Challenge

## 9. Programming exercises

For the final stage of the notebook you have four little programming exercises (see end). There are many ways of coding solutions to each of these exercises. Some solutions will be a lot more efficient than others. Learning how to write efficient code is one of the challenges of Python.

For each program measure the runtime of your code. You can do this easily by the iPython %timeit command. The command measures the time it takes to execute a single line of code. So to time a block of code you can place the code you want to measure in a simple dummy function called test() and then use %timeit to time how long it takes to execute test(), e.g.

    def test():
        somecode
    
    %timeit test()

As an example, consider the task of generating and storing 10,000 random numbers. There are three example solutions below. The solutions are functionally equivalent but have different runtimes. Writing efficient code can make a huge difference to execution time.

Method 1 - Using a loop and preallocating storage using np.zeros

In [ ]:
def test():
    x = np.zeros(10000)
    for n in range(10000):
        x[n] = np.random.uniform(0,1)

%timeit test()

Method 2 - Using a loop and preallocating unassigned storage using np.empty

In [ ]:
def test():
    x=np.empty(10000)
    for n in range(10000):
        x[n] = np.random.uniform(0,1)

%timeit test()

Method 3 - Avoiding the loop by asking the numpy.random.uniform function to generate all the numbers in one call

In [ ]:
def test():
    x = np.random.uniform(0,1,10000)

%timeit test()

The table shows execution times in ms on a Windows machine using Python 3

|    | Method 1 | Method 2 | Method 3 |
| :- | :-: | :-: | :-: |
| Python 3 | 4.46 ms | 4.75 ms | 0.127 ms |


Write solutions to the problems below in any way you please. Measure the execution times of your code for each exercise. 

1) Consider all the multiplications A times B for integers A and B lying in the range 1 to 1000. How many of these end in the digit 7. (Hint: You may want to experiment with the modulus '%' operator.)

2) Using the np.random.rand function generate 1,000,000 random numbers in the range 0 to 1 and then find the smallest and largest difference between a pair of adjacent numbers in the list.

3) Find the integers A and B in the range 1 to 1000 for which A/B lies closest to pi/4.

4) A random number is selected between 0 and 1 (with a uniform distribution). If a sequence of 4 such numbers is independently generated what is the probability that the sequence will be in ascending order? Estimate the probability by performing the experiment 1,000,000 times.